# WCI (World Cybercrime Index) Recalculation
This notebook rebuilds the per-country WCI scores from the raw 2021 survey data in `data/wci_data.csv` and writes the results to `data/WCI_recacl.csv`.


In [2]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('data')
RAW_PATH = DATA_DIR / 'wci_data.csv'
OUT_PATH = DATA_DIR / 'WCI_recacl.csv'

df = pd.read_csv(RAW_PATH)
print('Shape:', df.shape)
print('Head:')
try:
    from IPython.display import display  # type: ignore
except Exception:
    display = lambda x: print(x.head() if hasattr(x, 'head') else x)
display(df.head(3))

n_unique = df['ResponseID'].nunique()
print('Unique ResponseID count (should be ~92):', n_unique)
# Quick presence checks for expected columns
expected_blocks = ['Technical','Attack','Data','Scams','Cash']
missing_any = []
for block in expected_blocks:
    for i in range(1,6):
        base = f'{block}{i}'
        imp = f'{block}{i}_impact'
        prof = f'{block}{i}_professional'
        tech = f'{block}{i}_techskill'
        for c in [base, imp, prof, tech]:
            if c not in df.columns:
                missing_any.append(c)

print('Missing exemplar columns (if any):', missing_any[:10], '... total missing:', len(missing_any))


Shape: (92, 108)
Head:


,ResponseID,Nationality,Residence,Technical1,Technical2,Technical3,Technical4,Technical5,Technical1_impact,Technical1_professional,...,Cash4_professional,Cash4_techskill,Cash5_impact,Cash5_professional,Cash5_techskill,Expert_crimetype,Expert_crimetype_other,Expert_region,Expert_region_other,Comments
0,R1,United Kingdom,United Kingdom,Ukraine,Russia,Brazil,Romania,Latvia,3,6,...,NaN,NaN,NaN,NaN,NaN,Technical products / services,NaN,No,NaN,NaN
1,R2,Australia,Prefer not to say,Russia,Ukraine,--,--,--,10,9,...,NaN,NaN,NaN,NaN,NaN,"Technical products / services,Attacks and exto...",NaN,No,NaN,NaN
2,R3,Australia,Australia,Russia,Ukraine,United States,--,--,8,9,...,5.0,5.0,NaN,NaN,NaN,"Attacks and extortions,Data/identity theft,Cas...",NaN,Yes (please list below),Asia Pacific,While the survey should capture the highlights...


Unique ResponseID count (should be ~92): 92
Missing exemplar columns (if any): [] ... total missing: 0


## Build long table of nominations per crime type
For each of Technical/Attack/Data/Scams/Cash, gather the 1–5 nominations with their `impact`, `professionalism`, and `techskill` into one long dataframe. Rows with empty/NA or `--` country are dropped.


In [3]:
def build_long(df: pd.DataFrame) -> pd.DataFrame:
    records = []
    type_map = [
        ('technical', 'Technical'),
        ('attack', 'Attack'),
        ('data', 'Data'),
        ('scams', 'Scams'),
        ('cash', 'Cash'),
    ]
    for _, row in df.iterrows():
        resp = row.get('ResponseID')
        nat = row.get('Nationality')
        res = row.get('Residence')
        for ctype, prefix in type_map:
            for pos in range(1, 6):
                country = row.get(f'{prefix}{pos}')
                if pd.isna(country) or str(country).strip() == '' or str(country).strip() == '--':
                    continue
                impact = row.get(f'{prefix}{pos}_impact')
                prof = row.get(f'{prefix}{pos}_professional')
                tech = row.get(f'{prefix}{pos}_techskill')
                # Allow numeric conversion; ignore if all three are NA
                if pd.isna(impact) and pd.isna(prof) and pd.isna(tech):
                    continue
                records.append({
                    'respID': resp,
                    'nationality': nat,
                    'residence': res,
                    'crimetype': ctype,
                    'position': pos,
                    'country': country,
                    'impact': pd.to_numeric(impact, errors='coerce'),
                    'professionalism': pd.to_numeric(prof, errors='coerce'),
                    'techskill': pd.to_numeric(tech, errors='coerce'),
                })
    long_df = pd.DataFrame.from_records(records)
    # Drop rows where country missing after processing (safety), and where all three metrics are NA
    long_df = long_df.dropna(subset=['country'])
    long_df = long_df[~long_df['country'].astype(str).isin(['', '--'])]
    all_na = long_df[['impact','professionalism','techskill']].isna().all(axis=1)
    long_df = long_df[~all_na].copy()
    return long_df

index_long = build_long(df)
print('index_long shape:', index_long.shape)
display(index_long.head())
print('Crime types present:', index_long['crimetype'].unique())


index_long shape: (1736, 9)


,respID,nationality,residence,crimetype,position,country,impact,professionalism,techskill
0,R1,United Kingdom,United Kingdom,technical,1,Ukraine,3.0,6.0,5.0
1,R1,United Kingdom,United Kingdom,technical,2,Russia,5.0,8.0,7.0
2,R1,United Kingdom,United Kingdom,technical,3,Brazil,7.0,5.0,5.0
3,R1,United Kingdom,United Kingdom,technical,4,Romania,4.0,6.0,6.0
4,R1,United Kingdom,United Kingdom,technical,5,Latvia,5.0,7.0,6.0


Crime types present: ['technical' 'attack' 'data' 'scams' 'cash']


## Counts and mean scores per (crime type, country)
- `noms_type`: number of nominations a country received for that crime type.
- Mean of `impact`, `professionalism`, `techskill`.


In [4]:
# Nomination counts
noms = (
    index_long
    .groupby(['crimetype','country'], as_index=False)
    .size()
    .rename(columns={'size':'noms_type'})
)

# Mean scores
means = (
    index_long
    .groupby(['crimetype','country'], as_index=False)[['impact','professionalism','techskill']]
    .mean()
    .rename(columns={
        'impact':'impact_mean',
        'professionalism':'prof_mean',
        'techskill':'tech_mean'
    })
)

# Join means with counts
scores = means.merge(noms, on=['crimetype','country'], how='left')

# Overall = average of the three means
scores['Overall'] = scores[['impact_mean','prof_mean','tech_mean']].mean(axis=1)

# Total respondents N
N = df['ResponseID'].nunique()
print('Total respondents N:', N)

# WCI score per type
scores['WCI_Score'] = scores['Overall'] * (scores['noms_type'] / N) * 10

# Rank within each crime type
scores = scores.sort_values(['crimetype','WCI_Score'], ascending=[True, False]).copy()
scores['Rank'] = scores.groupby('crimetype')['WCI_Score'].rank(method='dense', ascending=False).astype(int)

# Final packaging
final_cols = ['crimetype','Rank','noms_type','country','impact_mean','prof_mean','tech_mean','Overall','WCI_Score']
final = scores[final_cols].copy()
final = final.rename(columns={
    'country':'Country',
    'impact_mean':'Impact',
    'prof_mean':'Professionalism',
    'tech_mean':'Technical_skill'
})
display(final.head(10))

# Write to CSV
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
final.to_csv(OUT_PATH, index=False)
print('Wrote', OUT_PATH.resolve())


Total respondents N: 92


,crimetype,Rank,noms_type,Country,Impact,Professionalism,Technical_skill,Overall,WCI_Score
35,attack,1,83,Russia,9.289157,8.927711,8.831325,9.016064,81.340580
42,attack,2,56,Ukraine,8.464286,8.196429,8.357143,8.339286,50.760870
22,attack,3,30,"Korea, North",8.033333,7.033333,7.300000,7.455556,24.311594
7,attack,4,28,China,8.178571,7.821429,7.892857,7.964286,24.239130
44,attack,5,23,United States,7.434783,6.782609,7.000000,7.072464,17.681159
17,attack,6,14,Iran,6.785714,6.428571,6.500000,6.571429,10.000000
34,attack,7,13,Romania,6.307692,6.307692,6.846154,6.487179,9.166667
4,attack,8,12,Brazil,7.250000,6.250000,6.666667,6.722222,8.768116
30,attack,9,11,Nigeria,7.545455,6.727273,6.818182,7.030303,8.405797
1,attack,10,7,Belarus,7.285714,7.428571,7.285714,7.333333,5.579710


Wrote /Users/user/codeprojects/wci/data/WCI_recacl.csv


### Optional: build a wide combined table of per-type WCI scores
This creates one row per country with columns `wci_tech`, `wci_attack`, `wci_data`, `wci_scams`, `wci_cash`.


In [5]:
wide = (
    final.pivot_table(
        index='Country',
        columns='crimetype',
        values='WCI_Score',
        aggfunc='first'
    )
    .rename(columns={
        'technical':'wci_tech',
        'attack':'wci_attack',
        'data':'wci_data',
        'scams':'wci_scams',
        'cash':'wci_cash'
    })
    .reset_index()
)
display(wide.head())


crimetype,Country,wci_attack,wci_cash,wci_data,wci_scams,wci_tech
0,Afghanistan,NaN,1.739130,NaN,NaN,NaN
1,Algeria,NaN,NaN,1.702899,NaN,NaN
2,Angola,0.688406,1.050725,NaN,NaN,NaN
3,Argentina,NaN,0.724638,NaN,NaN,NaN
4,Armenia,NaN,1.086957,0.978261,NaN,NaN
